In [2]:
# !pip install fosforml

In [2]:
# pip show fosforml

## Sklearn Models Registration

### session creation

In [6]:
from fosforml.model_manager.snowflakesession import get_session
session = get_session()

### modeling

#### Multiclass classification

In [7]:
from sklearn.ensemble._forest import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pandas as pd

m_dataset = load_iris() 
feature_names = m_dataset.feature_names
u_features = [col.replace(" ","_").replace("(","").replace(")","") for col in feature_names]
m_df = pd.DataFrame(m_dataset.data,columns=u_features)
m_df["Target"] = m_dataset.target


x_train,x_test,y_train,y_test = train_test_split(m_df.iloc[:,:-1],m_df['Target'])
model = RandomForestClassifier()
model.fit(x_train,y_train)

y_pred = pd.DataFrame(model.predict(x_test),columns=["Predicted"])

## Custom Function

In [10]:
from snowflake.ml.model import custom_model
from snowflake.ml.model.custom_model import Mo

mc = custom_model.ModelContext(
    models={
        'm1': model
    }
)

ImportError: cannot import name 'ModelContext' from 'snowflake.ml.model' (/opt/conda/lib/python3.9/site-packages/snowflake/ml/model/__init__.py)

In [ ]:
class ExamplePipelineModel(custom_model.CustomModel):
    def __init__(self, context: ModelContext) -> None:
        super().__init__(context)
        v = int(open(context.path('config')).read())
        self.bias = json.loads(v)['bias']

    @custom_model.inference_api
    def predict(self, input: pd.DataFrame) -> pd.DataFrame:
        features = self.context.model_ref('feature_preproc').transform(input)
        model_output = self.context.model_ref('m2').predict(
            self.context.model_ref('m1').predict(features)
        )
        return pd.DataFrame({
            'output': model_output + self.bias})

## Model registration

In [7]:
from fosforml import register_model
register_model(
  model_obj=mc,
  session=session,
  x_train=x_train,
  y_train=y_train,
  x_test=x_test,
  y_test=y_test,
  y_pred=y_pred,
  dataset_name="RandomMulticlassMOdelForTesting",
  dataset_source="Snowflake",
  name="RandomMultieefefefclassMOdelForTesting16July",
  description="This is a test sklearn model",
  flavour="sklearn",
  model_type="classification",
  conda_dependencies=["scikit-learn==1.3.2"]
)

Calculating build time metrics

Progress: ███████████████████████                                                33.3%


The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn<1.4'. Your UDF might not work when the package version is different between the server and your local environment.
DataFrame.flatten() is deprecated since 0.7.0. Use `DataFrame.join_table_function()` instead.
The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn<1.4'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn<1.4'. Your UDF might not work when the package version is different between the server and your local environment.


Calculating build time metrics

Progress: ███████████████████████████████████████████████                        66.7%
Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%


/opt/conda/lib/python3.9/contextlib.py:119: UserWarning: `relax_version` is not set and therefore defaulted to True. Dependency version constraints relaxed from ==x.y.z to >=x.y, <(x+1). To use specific dependency versions for compatibility, reproducibility, etc., set `options={'relax_version': False}` when logging the model.
  return next(self.gen)
/opt/conda/lib/python3.9/site-packages/snowflake/ml/model/model_signature.py:69: UserWarning: The sample input has 112 rows, thus a truncation happened before inferring signature. This might cause inaccurate signature inference. If that happens, consider specifying signature manually.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:913: RuntimeWarning: divide by zero encountered in log
  return np.log(proba)
/opt/conda/lib/python3.9/site-packages/snowflake/snowpark/session.py:2372: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.base.Index'> which will not be writt

Failed to set tags for model MODEL_216F1A62_8359_453E_AB91_1D36E047B5A9_FDC_RANDOMMULTIEEFEFEFCLASSMODELFORTESTING16JULY.
(1304): 01b5b59c-0002-a40e-0033-ac8700a2d1b2: 000002 (0A000): Unsupported feature 'TAG'.


"Model 'MODEL_216F1A62_8359_453E_AB91_1D36E047B5A9_FDC_RANDOMMULTIEEFEFEFCLASSMODELFORTESTING16JULY' registered successfully."